In [1]:
import random
import itertools
import numpy
import math
import sys

In [2]:
def calcular_matriz_de_costes(dataset):
    n_ciudades = len(dataset)
    distancias = numpy.zeros((n_ciudades,n_ciudades))
    for i in range(n_ciudades):
        for j in range(n_ciudades):
            if(i != j):
                distancias[i][j] = calcular_distancia_euclidea(dataset[i],dataset[j])
    return distancias

In [3]:
def calcular_distancia_euclidea(ciudad_1,ciudad_2):
    
    x1,y1 = ciudad_1[1 :]
    x2,y2 = ciudad_2[1 :]
    
    xd = x1 - x2
    yd = y1 - y2
    
    return round(math.sqrt( xd*xd + yd*yd ))

In [4]:
def calcular_coste(camino,distancias):
    coste = 0
    for indice in range(len(camino)-1):
        coste += distancias[camino[indice]][camino[indice+1]]
    coste += distancias[camino[-1]][camino[0]]
    return coste

In [5]:
def greedy(ciudad_inicial,dataset):
    
    ciudades_analizadas = [dataset[ciudad_inicial-1]]
    ciudades_no_analizadas = dataset.copy()
    ciudades_no_analizadas.pop(ciudad_inicial-1)
    
    while(len(ciudades_no_analizadas) > 0):
        
        distancia = sys.maxsize
        ciudad_mas_cercana = -1
        
        for indice in range(len(ciudades_no_analizadas )):
            
            distancia_calculada = calcular_distancia_euclidea(ciudades_analizadas[-1],ciudades_no_analizadas[indice])
            
            if(distancia > distancia_calculada):
                distancia = distancia_calculada
                ciudad_mas_cercana = indice
                
        ciudades_analizadas.append(ciudades_no_analizadas[ciudad_mas_cercana])
        ciudades_no_analizadas.pop(ciudad_mas_cercana)
   
    return ciudades_analizadas

In [6]:
def calcular_coste_greedy(camino):
    coste = 0
    for indice in range(len(camino)):
        if(indice < len(camino)-1):
            coste += calcular_distancia_euclidea(camino[indice],camino[indice+1])
        else:
            coste += calcular_distancia_euclidea(camino[indice],camino[0])
    return coste

In [7]:
def generar_array_ciudades_cercanas_a_vecinos(combinaciones_indices_vecinos,len_dataset):
    ciudades_cercanas = []

    for combinacion in combinaciones_indices_vecinos:
        if (combinacion[0] == 0):
            a1 = len_dataset - 1
        else:
            a1 = combinacion[0] - 1

        if (combinacion[0] == len_dataset - 1):
            s1 = 0
        else:
            s1 = combinacion[0] + 1

        if (combinacion[1] == 0):
            a2 = len_dataset - 1
        else:
            a2 = combinacion[1] - 1

        if (combinacion[1] == len_dataset - 1):
            s2 = 0
        else:
            s2 = combinacion[1] + 1

        ciudades_cercanas.append([a1, s1, a2, s2])

    return ciudades_cercanas

In [8]:
def calcular_coste_intercambio(mejor_coste,mejor_solucion,vecinos,intercambio,len_dataset,matriz_de_costes):
    # Por cada vecino, estudiaremos su coste
    coste_candidato = mejor_coste

    # Primero, por comodidad, generaremos los indices del camino
    a1 = vecinos[0]
    s1 = vecinos[1]
    a2 = vecinos[2]
    s2 = vecinos[3]
    i1 = intercambio[0]
    i2 = intercambio[1]

    if (i2 - 1 == i1):

        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s2]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s2]]

    elif(i1 == 0 and i2 == len_dataset-1):
        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s1]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s1]]
    else:

        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s1]]

        coste_candidato -= matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s2]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s1]]

        coste_candidato += matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s2]]
    
    return coste_candidato

In [9]:
def busqueda_tabu(semilla, dataset,num_vecinos):
    
    random.seed(semilla)                                                # Se establece la semilla inicial
    len_dataset = len(dataset)                                          # Tamaño dataset
    matriz_de_costes = calcular_matriz_de_costes(dataset)               # Generación de matriz de costes
    solucion_actual = random.sample(range(len_dataset),len_dataset)     # Generación de solución inicial
    coste_actual = calcular_coste(solucion_actual, matriz_de_costes)     # Generación del coste inicial

    # Variables de la busqueda tabu
    lista_tabu = []
    lista_tabu_size = len_dataset/2
    lista_tabu_cabeza = 0
    matriz_de_frecuencia = numpy.zeros((len_dataset,len_dataset))
    mejor_solucion = solucion_actual.copy()
    mejor_coste = coste_actual

    # Indices de todos los posibles intercambios de dos ciudades del dataset [ (0,1), (0,2), ... , (10,23), ...  ]
    intercambios_de_dos_ciudades = list(itertools.combinations(list(range(0, len_dataset)), 2))
    len_intercambios_de_dos_ciudades = len(intercambios_de_dos_ciudades)

    # Indices de los vecinos de los indices del array intercambios_de_dos_ciudades : 
    # intercambios_de_dos_ciudades[x] = (6,10)
    # vecinos_de_intercambios_de_dos_ciudades = [5,7,9,11]
    vecinos_de_intercambios_de_dos_ciudades = generar_array_ciudades_cercanas_a_vecinos(intercambios_de_dos_ciudades,len_dataset)

    
    seguir = True       # Variable de salida del bucle
    evaluacion = 1      # Nº de evaluaciones actual del bucle
    iteraciones = 0     # Nº de interaciones del bucle

    # Condición de parada del bucle
    num_iteraciones = 40 * len_dataset

    while (seguir):

        indices = random.sample(range(len_intercambios_de_dos_ciudades), num_vecinos)
        mejor_vecino = []
        coste_mejor_vecino = sys.maxsize


        for i in indices:

            coste_candidato = calcular_coste_intercambio(coste_actual,solucion_actual,vecinos_de_intercambios_de_dos_ciudades[i],intercambios_de_dos_ciudades[i],len_dataset,matriz_de_costes)

            evaluacion += 1

            tabu = False

            # Se comprueba si el vecino esta en la lista tabu
            if( any( combinacion == intercambios_de_dos_ciudades[i] for combinacion in lista_tabu ) ):
                # Prueba de aspiración 
                if coste_candidato >= mejor_coste:
                    tabu = True
            
            if(not tabu):
                if coste_candidato < coste_mejor_vecino:
                    coste_mejor_vecino = coste_candidato
                    mejor_vecino = intercambios_de_dos_ciudades[i]
        

        if(not (mejor_vecino == [])):
            buffer = solucion_actual[mejor_vecino[0]]
            solucion_actual[mejor_vecino[0]] = solucion_actual[mejor_vecino[1]]
            solucion_actual[mejor_vecino[1]] = buffer
            coste_actual = coste_mejor_vecino

            if coste_actual < mejor_coste:
                mejor_solucion = solucion_actual.copy()
                mejor_coste = coste_actual
            
            # Añadimos a la lista tabu el movimiento
            if(lista_tabu_cabeza < lista_tabu_size):
                
                if(len(lista_tabu) < lista_tabu_size):
                    lista_tabu.append(mejor_vecino)
                else:
                    lista_tabu[lista_tabu_cabeza] = mejor_vecino

                lista_tabu_cabeza+=1

            else:
                lista_tabu_cabeza = 1
                lista_tabu[0] = mejor_vecino

            # Actualizamos la matriz de frecuencia
            for indice in range(len_dataset):
                matriz_de_frecuencia[indice,solucion_actual[indice]] += 1

        iteraciones += 1
        # Reinicio
        if(iteraciones % (8 * len_dataset ) == 0 and iteraciones != num_iteraciones):
            
            lista_tabu = []
            lista_tabu_cabeza = 0

            if(random.uniform(0,1) < 0.5):
                lista_tabu_size += lista_tabu_size/2
            else:
                lista_tabu_size -= lista_tabu_size/2

            aleatorio = random.uniform(0,1)
            if(aleatorio <= 0.25):

                # Solución actual aleatoria
                solucion_actual = random.sample(range(len_dataset),len_dataset)
                coste_actual = calcular_coste(solucion_actual, matriz_de_costes)

            elif(aleatorio <= 0.5):

                # Intensificamos
                solucion_actual = mejor_solucion.copy()
                coste_actual = mejor_coste.copy()
                evaluacion += 1

            else:
                
                # greedy (1º implementación)
                lista_analizar = list(range(len_dataset))
                solucion_actual = []
                for lista in matriz_de_frecuencia:
                    frecuencia = sys.maxsize
                    posicion = -1
                    for indice in lista_analizar:
                        if( lista[indice] == 0 ):
                            posicion = indice
                            break
                        else:
                            if(lista[indice] < frecuencia):
                                frecuencia = lista[indice]
                                posicion = indice
                    solucion_actual.append(posicion)
                    lista_analizar.remove(posicion)
                coste_actual = calcular_coste(solucion_actual, matriz_de_costes)
                evaluacion += 1
                    
                            
        # Salida del bucle
        if iteraciones == num_iteraciones:
            seguir = False

    return [dataset[ciudad] for ciudad in mejor_solucion], mejor_coste, evaluacion